# Example Seldon Core Deployments using Helm
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)
 - [Helm client](https://helm.sh/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

# Setup

In [1]:
!kubectl create namespace seldon

namespace/seldon created


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


In [3]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


# Install Helm

In [4]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount/tiller created
clusterrolebinding.rbac.authorization.k8s.io/tiller created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [5]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


## Start seldon-core

In [9]:
!helm install ../../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

NAME:   seldon-core-crd
LAST DEPLOYED: Fri Oct 19 18:30:11 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     0s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          0s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s


NOTES:
NOTES: TODO




In [12]:
!helm install ../../helm-charts/seldon-core --name seldon-core --namespace seldon  --set ambassador.enabled=true

NAME:   seldon-core
LAST DEPLOYED: Fri Oct 19 18:33:43 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/ServiceAccount
NAME    SECRETS  AGE
seldon  1        0s

==> v1beta1/ClusterRole
NAME        AGE
seldon-crd  0s

==> v1/ClusterRoleBinding
NAME    AGE
seldon  0s

==> v1beta1/Role
NAME          AGE
seldon-local  0s
ambassador    0s

==> v1beta1/RoleBinding
NAME        AGE
ambassador  0s

==> v1beta1/Deployment
NAME                                DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-core-ambassador              1        1        1           0          0s
seldon-core-seldon-apiserver        1        1        1           0          0s
seldon-core-seldon-cluster-manager  1        1        1           0          0s
seldon-core-redis                   1        1        1           0          0s

==> v1/RoleBinding
NAME    AGE
seldon  0s

==> v1/Service
NAME                          TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)                        AGE
seldon-co

In [13]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador

deployment "seldon-core-seldon-cluster-manager" successfully rolled out
deployment "seldon-core-seldon-apiserver" successfully rolled out
Waiting for deployment "seldon-core-ambassador" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-ambassador" successfully rolled out


## Set up REST and gRPC methods

**Ensure you port forward ambassador and API gateway**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8002:8080
```

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8004:5000
```

Install gRPC modules for the prediction protos.

In [14]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

cp: cannot stat '../proto/prediction.proto': No such file or directory
./proto/prediction.proto: No such file or directory


## Serve Single Model

In [16]:
!helm install ../../helm-charts/seldon-single-model --name mymodel --set oauth.key=oauth-key --set oauth.secret=oauth-secret

NAME:   mymodel
LAST DEPLOYED: Fri Oct 19 18:35:50 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME     AGE
mymodel  0s




In [17]:
!helm template ../../helm-charts/seldon-single-model --name mymodel --set oauth.key=oauth-key --set oauth.secret=oauth-secret | pygmentize -l json

---
# Source: seldon-single-model/templates/model.json
{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "mymodel"
    },
    "spec": {
        "name": "mymodel",        
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                                }
                            

In [18]:
!kubectl rollout status deploy/mymodel-mymodel-classifier-0
!kubectl rollout status deploy/mymodel-mymodel-svc-orch

Waiting for deployment "mymodel-mymodel-classifier-0" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymodel-mymodel-classifier-0" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymodel-mymodel-classifier-0" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymodel-mymodel-classifier-0" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymodel-mymodel-classifier-0" successfully rolled out
deployment "mymodel-mymodel-svc-orch" successfully rolled out


### Get predictions

In [22]:
import sys
sys.path.append("../../notebooks")
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [23]:
r = rest_request_api_gateway("oauth-key","oauth-secret")
print(r.text)

{"access_token":"7892c1ce-ab4b-43b8-a836-fbf872d7dba9","token_type":"bearer","expires_in":43199,"scope":"read write"}
{
  "meta": {
    "puid": "5qeb57bpbj3p82tjc0nb9v2s0q",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.0"
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.07824180738722948]
    }
  }
}


In [24]:
r = rest_request_ambassador("mymodel",API_AMBASSADOR)
print(r.text)

{
  "meta": {
    "puid": "ikg7ft60992a1tgk5k3eceogmg",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.0"
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.0871649338678916]
    }
  }
}


#### gRPC Request

In [25]:
grpc_request_ambassador("mymodel",API_AMBASSADOR)

meta {
  puid: "me6t461pjhk2gotkif3qmdbv9i"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.06833286694600807
  }
}

In [26]:
grpc_request_api_gateway("oauth-key","oauth-secret",grpc_endpoint="localhost:8004")

{"access_token":"7892c1ce-ab4b-43b8-a836-fbf872d7dba9","token_type":"bearer","expires_in":43190,"scope":"read write"}


meta {
  puid: "oi30b58mika5ismlnjktpnda8n"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.09278521594160527
  }
}

In [27]:
!helm delete mymodel --purge

release "mymodel" deleted


## Serve AB Test

In [29]:
!helm install ../../helm-charts/seldon-abtest --name myabtest --set oauth.key=oauth-key --set oauth.secret=oauth-secret

NAME:   myabtest
LAST DEPLOYED: Fri Oct 19 18:45:42 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME      AGE
myabtest  0s




In [31]:
!helm template ../../helm-charts/seldon-abtest --name myabtest --set oauth.key=oauth-key --set oauth.secret=oauth-secret | pygmentize -l json

---
# Source: seldon-abtest/templates/ab_test_1pod.json


---
# Source: seldon-abtest/templates/ab_test_2pods.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
	"labels": {
	    "app": "seldon"
	},
	"name": "myabtest"
    },
    "spec": {
	"name": "myabtest",        
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
	"predictors": [
	    {
		"name": "abtest",
		"replicas": 1,
		"componentSpecs": [{
		    "spec": {
			"containers": [
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-1",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }],
			"terminationGracePeriodSeconds": 20
		    }},
	        {
		    "metadata":{
			"labels":{
			    "version":"v2"
			}
		    },    
			"spec":{
			    "containers":[
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"

In [32]:
!kubectl rollout status deploy/myabtest-abtest-classifier-1-0
!kubectl rollout status deploy/myabtest-abtest-classifier-1-0
!kubectl rollout status deploy/myabtest-abtest-svc-orch 

deployment "myabtest-abtest-classifier-1-0" successfully rolled out
deployment "myabtest-abtest-classifier-1-0" successfully rolled out
deployment "myabtest-abtest-svc-orch" successfully rolled out


### Get predictions

#### REST Request

In [33]:
r = rest_request_ambassador("myabtest",API_AMBASSADOR)
print(r.text)

{
  "meta": {
    "puid": "ifcr21qadvou58dmc76kgiu24v",
    "tags": {
    },
    "routing": {
      "myabtest": 1
    },
    "requestPath": {
      "myabtest": "",
      "classifier-2": "seldonio/mock_classifier:1.0"
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.0849806651591639]
    }
  }
}


In [34]:
r = rest_request_api_gateway("oauth-key","oauth-secret")
print(r.text)

{"access_token":"7892c1ce-ab4b-43b8-a836-fbf872d7dba9","token_type":"bearer","expires_in":43076,"scope":"read write"}
{
  "meta": {
    "puid": "chg6ps28i2hnc6oa4dptcra4ph",
    "tags": {
    },
    "routing": {
      "myabtest": 0
    },
    "requestPath": {
      "myabtest": "",
      "classifier-1": "seldonio/mock_classifier:1.0"
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.10304073265496357]
    }
  }
}


#### gRPC Request

In [35]:
grpc_request_ambassador("myabtest",API_AMBASSADOR)

meta {
  puid: "mfd2023j1a3ethn5j5o9o5k2jf"
  routing {
    key: "myabtest"
    value: 1
  }
  requestPath {
    key: "classifier-2"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "myabtest"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.07409465499499068
  }
}

In [36]:
grpc_request_api_gateway("oauth-key","oauth-secret",grpc_endpoint="localhost:8004")

{"access_token":"7892c1ce-ab4b-43b8-a836-fbf872d7dba9","token_type":"bearer","expires_in":43050,"scope":"read write"}


meta {
  puid: "9egs54q5lotncnqknj9gbdobei"
  routing {
    key: "myabtest"
    value: 1
  }
  requestPath {
    key: "classifier-2"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "myabtest"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08308652534451443
  }
}

In [37]:
!helm delete myabtest --purge

release "myabtest" deleted


## Serve Multi-Armed Bandit

In [39]:
!helm install ../../helm-charts/seldon-mab --name mymab --set oauth.key=oauth-key --set oauth.secret=oauth-secret

NAME:   mymab
LAST DEPLOYED: Fri Oct 19 18:47:57 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME   AGE
mymab  0s




In [40]:
!helm template ../../helm-charts/seldon-mab --name mymab --set oauth.key=oauth-key --set oauth.secret=oauth-secret | pygmentize -l json

---
# Source: seldon-mab/templates/mab.json
{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
	"labels": {
	    "app": "seldon"
	},
	"name": "mymab"
    },
    "spec": {
	"name": "mymab",        
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
	"predictors": [
	    {
		"name": "abtest",
		"replicas": 1,
		"componentSpecs": [{
		    "spec": {
			"containers": [
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-1",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }],
			"terminationGracePeriodSeconds": 20
		    }},
	        {
		    "metadata":{
			"labels":{
			    "version":"v2"
			}
		    },    
			"spec":{
			    "containers":[
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-2",
				"resources": {
				    "requests": {
					"memory":

In [41]:
!kubectl rollout status deploy/mymab-abtest-classifier-1-0
!kubectl rollout status deploy/mymab-abtest-classifier-2-1
!kubectl rollout status deploy/mymab-abtest-eg-router-2 

Waiting for deployment "mymab-abtest-classifier-1-0" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymab-abtest-classifier-1-0" successfully rolled out
deployment "mymab-abtest-classifier-2-1" successfully rolled out
Waiting for deployment "mymab-abtest-eg-router-2" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymab-abtest-eg-router-2" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymab-abtest-eg-router-2" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymab-abtest-eg-router-2" rollout to finish: 0 of 1 updated replicas are available...

### Get predictions

#### REST Request

In [42]:
r = rest_request_ambassador("mymab",API_AMBASSADOR)
print(r.text)

{
  "meta": {
    "puid": "4a9p29vil4nitor6vn18mmcgjv",
    "tags": {
    },
    "routing": {
      "eg-router": 0
    },
    "requestPath": {
      "eg-router": "seldonio/mab_epsilon_greedy:1.1",
      "classifier-1": "seldonio/mock_classifier:1.0"
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.09101330616093926]
    }
  }
}


In [43]:
r = rest_request_api_gateway("oauth-key","oauth-secret")
print(r.text)

{"access_token":"7892c1ce-ab4b-43b8-a836-fbf872d7dba9","token_type":"bearer","expires_in":42796,"scope":"read write"}
{
  "meta": {
    "puid": "p8kobe9ap7uhols611194th32d",
    "tags": {
    },
    "routing": {
      "eg-router": 0
    },
    "requestPath": {
      "eg-router": "seldonio/mab_epsilon_greedy:1.1",
      "classifier-1": "seldonio/mock_classifier:1.0"
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.1002378266885396]
    }
  }
}


#### gRPC Request

In [44]:
grpc_request_ambassador("mymab",API_AMBASSADOR)

meta {
  puid: "vpomgkhv3v37vm0vg4sqip6i05"
  routing {
    key: "eg-router"
  }
  requestPath {
    key: "classifier-1"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "eg-router"
    value: "seldonio/mab_epsilon_greedy:1.1"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.07252885966272392
  }
}

In [45]:
grpc_request_api_gateway("oauth-key","oauth-secret",grpc_endpoint="localhost:8004")

{"access_token":"7892c1ce-ab4b-43b8-a836-fbf872d7dba9","token_type":"bearer","expires_in":42772,"scope":"read write"}


meta {
  puid: "9grr0vjkscmg1a28pqc6tl34tl"
  routing {
    key: "eg-router"
  }
  requestPath {
    key: "classifier-1"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "eg-router"
    value: "seldonio/mab_epsilon_greedy:1.1"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.07922567094086788
  }
}

In [46]:
!helm delete mymab --purge

release "mymab" deleted


## Serve Single Model with Outlier Detector

In [47]:
!helm install ../../helm-charts/seldon-single-model --name mymodel --set outlier_detection.enabled=true --set oauth.key=oauth-key --set oauth.secret=oauth-secret

NAME:   mymodel
LAST DEPLOYED: Fri Oct 19 18:52:39 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME     AGE
mymodel  0s




In [48]:
!helm template ../../helm-charts/seldon-single-model --name mymodel --set outlier_detection.enabled=true --set oauth.key=oauth-key --set oauth.secret=oauth-secret | pygmentize -l json

---
# Source: seldon-single-model/templates/model.json
{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "mymodel"
    },
    "spec": {
        "name": "mymodel",        
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                                }
                            

In [49]:
!kubectl rollout status deploy/mymodel-mymodel-classifier-0
!kubectl rollout status deploy/mymodel-mymodel-svc-orch
!kubectl rollout status deploy/mymodel-mymodel-outlier-detector-1

deployment "mymodel-mymodel-classifier-0" successfully rolled out
Waiting for deployment "mymodel-mymodel-svc-orch" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymodel-mymodel-svc-orch" successfully rolled out
Waiting for deployment "mymodel-mymodel-outlier-detector-1" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymodel-mymodel-outlier-detector-1" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymodel-mymodel-outlier-detector-1" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymodel-mymodel-outlier-detector-1" rollout to finish: 0 of 

### Get predictions

#### REST Request

In [50]:
r = rest_request_ambassador("mymodel",API_AMBASSADOR)
print(r.text)

{
  "meta": {
    "puid": "a1m9eqjalqu2vtg2fcv24iutqv",
    "tags": {
      "outlierScore": [0.0]
    },
    "routing": {
      "outlier-detector": -1
    },
    "requestPath": {
      "outlier-detector": "seldonio/outlier_mahalanobis:0.3",
      "classifier": "seldonio/mock_classifier:1.0"
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.09249787999217929]
    }
  }
}


In [51]:
r = rest_request_api_gateway("oauth-key","oauth-secret")
print(r.text)

{"access_token":"7892c1ce-ab4b-43b8-a836-fbf872d7dba9","token_type":"bearer","expires_in":42667,"scope":"read write"}
{
  "meta": {
    "puid": "sb5c4vdclkoihsumghjmm8jkfo",
    "tags": {
      "outlierScore": [0.0]
    },
    "routing": {
      "outlier-detector": -1
    },
    "requestPath": {
      "outlier-detector": "seldonio/outlier_mahalanobis:0.3",
      "classifier": "seldonio/mock_classifier:1.0"
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.07854121999267133]
    }
  }
}


#### gRPC Request

In [52]:
grpc_request_ambassador("mymodel",API_AMBASSADOR)

meta {
  puid: "hnr03ujhprah78b2lrbd418u59"
  tags {
    key: "outlierScore"
    value {
      list_value {
        values {
          number_value: 0.0
        }
      }
    }
  }
  routing {
    key: "outlier-detector"
    value: -1
  }
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "outlier-detector"
    value: "seldonio/outlier_mahalanobis:0.3"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.0861506710600797
  }
}

In [53]:
grpc_request_api_gateway("oauth-key","oauth-secret",grpc_endpoint="localhost:8004")

{"access_token":"7892c1ce-ab4b-43b8-a836-fbf872d7dba9","token_type":"bearer","expires_in":42650,"scope":"read write"}


meta {
  puid: "e4aqe6s5ei8885o92804knc3n2"
  tags {
    key: "outlierScore"
    value {
      list_value {
        values {
          number_value: 191.44918593397708
        }
      }
    }
  }
  routing {
    key: "outlier-detector"
    value: -1
  }
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "outlier-detector"
    value: "seldonio/outlier_mahalanobis:0.3"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.11002593453960169
  }
}

In [54]:
!helm delete mymodel --purge

release "mymodel" deleted
